In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from datetime import datetime

#Torch-related imports
import torch
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function

#Model-specific imports
from SBM_SDE import *
from obs_and_flow import *
from training import *

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

In [3]:
#Neural SDE parameters
dt_flow = 0.2
t = 380 #In hours.
n = int(t / dt_flow) + 1
t_span = np.linspace(0, t, n)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [4]:
#SBM temperature forcing parameters
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.

In [5]:
#Training parameters
niter = 8500
piter = 200
pretrain_lr = 1e-2 #Norm regularization learning rate
train_lr = 1e-3 #ELBO learning rate
batch_size = 5
obs_error_scale = 0.1 #Observation (y) standard deviation
num_layers = 5

In [6]:
#SBM prior means
#System parameters from deterministic CON model
u_M = 0.0009
a_SD = 0.5
a_DS = 0.5
a_M = 0.5
a_MSC = 0.5
k_S_ref = 0.0001
k_D_ref = 0.001
k_M_ref = 0.00022
Ea_S = 55
Ea_D = 48
Ea_M = 48

#SCON diffusion matrix parameters
c_SOC = 0.05
c_DOC = 0.001
c_MBC = 0.0005

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [7]:
#Initial condition prior means
x0_SCON = [65, 0.4, 2.5]
x0_SCON_tensor = torch.tensor(x0_SCON)
x0_prior_SCON = D.multivariate_normal.MultivariateNormal(x0_SCON_tensor,
                                                         scale_tril=torch.eye(state_dim_SCON) * obs_error_scale * x0_SCON_tensor)

In [8]:
#Generate exogenous input vectors.
#Obtain temperature forcing function.
temp_tensor = temp_gen(t_span_tensor, temp_ref, temp_rise)

#Obtain SOC and DOC pool litter input vectors for use in flow SDE functions.
i_s_tensor = i_s(t_span_tensor) #Exogenous SOC input function
i_d_tensor = i_d(t_span_tensor) #Exogenous DOC input function

In [ ]:
#Call training loop function for SCON-C.
net, ELBO_hist = train(devi, pretrain_lr, train_lr, niter, piter, batch_size, num_layers,
          state_dim_SCON, 'y_from_x_t_5000_dt_0-01.csv', obs_error_scale, t, dt_flow, n, 
          t_span_tensor, i_s_tensor, i_d_tensor, temp_tensor, temp_ref,
          drift_diffusion_SCON_C, x0_prior_SCON, SCON_C_params_dict,
          LEARN_PARAMS = False, LR_DECAY = 0.1, DECAY_STEP_SIZE = 1000, PRINT_EVERY = 10)


Train Diffusion:   0%|          | 9/8500 [00:34<8:57:13,  3.80s/it]

Moving average norm loss at 10 iterations is: 574289.40625. Best norm loss value is: 569144.0.

C_PATH mean = tensor([[1.4580, 1.4391, 1.4446],
        [1.4558, 1.4308, 1.4476],
        [1.4609, 1.4371, 1.4465],
        [1.4580, 1.4330, 1.4540],
        [1.4595, 1.4429, 1.4484]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[2.8990, 2.6534, 2.0441],
         [1.9403, 1.9199, 2.4904],
         [2.8834, 1.8115, 2.7542],
         ...,
         [1.3151, 1.3382, 1.3012],
         [1.1771, 1.8702, 1.3909],
         [1.3950, 1.5578, 1.3693]],

        [[1.8498, 2.9279, 2.7291],
         [2.5502, 2.5542, 1.6569],
         [1.8473, 2.1055, 2.4058],
         ...,
         [1.0129, 1.6903, 1.2076],
         [1.3562, 1.1230, 1.1481],
         [1.5341, 1.4143, 1.2733]],

        [[1.4698, 2.9151, 2.3121],
         [2.2997, 2.4588, 2.3552],
         [2.0502, 2.3390, 2.1276],
         ...,
         [1.2437, 1.2078, 1.7167],
         [1.7339, 1.6161, 1.6266],
         [1.0161, 1.1537, 1.1427]],

      


Train Diffusion:   0%|          | 19/8500 [01:13<9:01:56,  3.83s/it]

Moving average norm loss at 20 iterations is: 568562.7875. Best norm loss value is: 566608.75.

C_PATH mean = tensor([[1.9186, 1.7892, 1.8502],
        [1.9032, 1.7911, 1.8578],
        [1.9100, 1.8090, 1.8476],
        [1.9115, 1.8099, 1.8594],
        [1.8942, 1.7964, 1.8472]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[2.3649, 2.8337, 1.5357],
         [1.5932, 2.2296, 1.6302],
         [2.0676, 1.8597, 1.8569],
         ...,
         [2.1487, 2.1943, 1.3981],
         [1.1468, 2.1185, 2.5933],
         [2.2006, 1.9887, 1.6791]],

        [[2.0965, 2.4690, 2.1772],
         [2.0745, 1.5281, 1.5711],
         [2.3918, 1.7500, 1.6617],
         ...,
         [2.2539, 1.7611, 2.4230],
         [2.8070, 1.8475, 1.8656],
         [1.2211, 2.0449, 2.1457]],

        [[1.4306, 2.4641, 1.9059],
         [2.0308, 2.0291, 1.2378],
         [1.9717, 1.4657, 1.5392],
         ...,
         [0.7752, 1.1721, 2.0339],
         [1.9704, 1.8947, 1.3108],
         [2.7678, 1.5360, 2.3439]],

      


Train Diffusion:   0%|          | 29/8500 [01:51<8:51:20,  3.76s/it]

...


Train Diffusion:   6%|▌         | 489/8500 [29:39<8:10:23,  3.67s/it]

Moving average ELBO loss at 490 iterations is: 7622534.85. Best ELBO loss value is: 5584225.5.

C_PATH mean = tensor([[54.8397,  0.8280,  2.5264],
        [54.4909,  0.8411,  2.5288],
        [54.6538,  0.8326,  2.5231],
        [54.6746,  0.8308,  2.5244],
        [54.4382,  0.8521,  2.5255]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[21.5126,  0.8685,  2.7830],
         [55.5360,  0.7167,  2.5551],
         [56.0900,  0.7604,  2.5628],
         ...,
         [54.7668,  0.8567,  2.5751],
         [55.5723,  0.8165,  2.5241],
         [56.0418,  0.8016,  2.3889]],

        [[21.7403,  0.3553,  2.5223],
         [54.9669,  0.8474,  2.6313],
         [54.1209,  0.6412,  2.6038],
         ...,
         [56.1943,  0.8898,  2.5165],
         [54.6063,  0.8299,  2.5680],
         [55.8887,  0.7806,  2.4941]],

        [[21.7486,  0.3882,  2.6115],
         [55.3571,  0.8173,  2.5219],
         [55.3543,  0.6941,  2.4652],
         ...,
         [56.4265,  0.8434,  2.4284],
         [55.7


Train Diffusion:   6%|▌         | 499/8500 [30:21<9:01:47,  4.06s/it]

Moving average ELBO loss at 500 iterations is: 7601561.75. Best ELBO loss value is: 5584225.5.

C_PATH mean = tensor([[54.3053,  0.8341,  2.5180],
        [54.3319,  0.8549,  2.5294],
        [54.6403,  0.8303,  2.5241],
        [54.6496,  0.8299,  2.5333],
        [54.5578,  0.8374,  2.5251]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[21.0558,  1.3940,  2.1398],
         [29.0401,  2.7347,  2.1243],
         [55.0433,  0.7769,  2.4313],
         ...,
         [53.6137,  0.8927,  2.5610],
         [55.7751,  0.6723,  2.4601],
         [49.1424,  0.5726,  2.5028]],

        [[21.3434,  0.8881,  2.2778],
         [54.0729,  1.2206,  2.4930],
         [55.3454,  0.8210,  2.5355],
         ...,
         [56.0508,  0.8927,  2.5753],
         [55.1925,  0.8407,  2.5796],
         [54.9245,  0.8916,  2.3672]],

        [[22.1033,  0.4029,  2.4984],
         [53.5006,  0.7539,  2.7841],
         [54.8400,  0.7652,  2.5841],
         ...,
         [55.4826,  0.8211,  2.4691],
         [54.4


Train Diffusion:   6%|▌         | 508/8500 [30:59<9:05:59,  4.10s/it]

In [ ]:
now = datetime.now()
now_string = now.strftime("%Y_%m_%d_%H_%M_%S")
torch.save(net, f'net_t_{t}_dt_{dt_flow}_iter_{niter}_{now_string}.pt')
torch.save(ELBO_hist, f'ELBO_t_{t}_dt_{dt_flow}_iter_{niter}_{now_string}.pt')